In [2]:
import time
    
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics

import os
import joblib
import matplotlib.pyplot as plt

from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import itertools



In [6]:
cici_test_dir='/home/irteam/junghye-dcloud-dir/MLAC/230624/CICI'
X_test=pd.read_csv(os.path.join(cici_test_dir,'X_test.csv'))
y_test=pd.read_csv(os.path.join(cici_test_dir,'y_test.csv'))

In [4]:
df=pd.DataFrame(columns=['L1_acc','L1_f1','L1_rc','L1_pc','L2_acc','L2_f1','L2_rc','L2_pc']+\
                 ['c1_acc','c1_f1','c1_rc','c1_pc']+\
                    ['c2_acc','c2_f1','c2_rc','c2_pc','c3_acc','c3_f1','c3_rc','c3_pc','c4_acc','c4_f1','c4_rc','c4_pc']+\
                     ['total_acc','total_f1','total_rc','total_pc'])

outpath='/home/irteam/junghye-dcloud-dir/MLAC/230624/CICI'
cnt=0
model_eval=[]

In [7]:
X_test.head()

,Unnamed: 0,protocol,flow_duration,tot_fwd_pkts,tot_bwd_pkts,tot_len_fwd_pkts,tot_len_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,fwd_pkt_len_mean,...,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,attack_category,nist_category
0,6170590,0.385385,0.279279,0.000000,0.370370,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,5847639,0.385385,0.528550,0.856356,0.871371,0.751251,0.897898,0.694194,0.000000,0.524024,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,1157416,0.385385,0.331832,0.482482,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,2038020,0.385385,0.443333,0.738739,0.700200,0.958580,0.779005,0.990885,0.000000,0.970421,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,684465,1.000000,0.725435,0.311311,0.136136,0.470971,0.600045,0.578579,0.894895,0.720721,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [8]:
L1_ytest=y_test.copy()

L2_ytest=X_test['nist_category'].copy()

L3_ytest=X_test['attack_category'].copy()

c1_Xtest=X_test.query('nist_category==1')
c1_ytest=c1_Xtest['attack_category']

c2_Xtest=X_test.query('nist_category==2')
c2_ytest=c2_Xtest['attack_category']

c3_Xtest=X_test.query('nist_category==3')
c3_ytest=c3_Xtest['attack_category']

c4_Xtest=X_test.query('nist_category==4')
c4_ytest=c4_Xtest['attack_category']

In [ ]:
for class_data in [c1_Xtest,c2_Xtest,c3_Xtest,c4_Xtest]:
    class_data.drop(labels=['attack_category','nist_category'],axis=1,inplace=True)

In [ ]:
X_test.drop(labels=['Unnamed: 0','nist_category','attack_category'],axis=1,inplace=True)

In [ ]:

def test_result(model:str,test,pred) ->list:

    acc=accuracy_score(test,pred)
    f1=f1_score(test,pred,average='weighted')
    recall=recall_score(test,pred,average='weighted')
    precision=precision_score(test,pred,average='weighted')

    print(f'{model} result , acc:{acc}, f1:{f1},recall:{recall},precision:{precision}')
    return([acc,f1,recall,precision])


In [ ]:
saved_path='/home/irteam/dcloud-global-dir/MLAC/saved_models/230620/savedmodels'
confusion_path='/home/irteam/junghye-dcloud-dir/MLAC/230624/CICI/confusion'

In [ ]:
L1_model=joblib.load(os.path.join(saved_path,'CICI_L1mod_rc.pkl'))



In [ ]:
L1_ypred=L1_model.predict(X_test)

eval_result=test_result(L1_model,L1_ytest,L1_ypred)


model_eval.extend(eval_result)

In [ ]:
confusion=metrics.confusion_matrix(L1_ytest,L1_ypred)
plot_confusion_matrix(confusion,labels=[0,1],title='Layer1')

In [ ]:
malicious_indices=np.where(L1_ypred==1)[0]


if malicious_indices.any():
    L2_model=joblib.load(os.path.join(saved_path,'CICI_nist.pkl'))
    L2_Xtest=X_test.iloc[malicious_indices]
    L2_ypred=L2_model.predict(L2_Xtest)
    L2_ytest_selected=L2_ytest.iloc[malicious_indices]
    L2_result=test_result(L2_model,L2_ytest_selected,L2_ypred)
    
    model_eval.extend(L2_result)
else:
    print('no malicious predicted')
    import sys
    sys.exit()
    

In [ ]:
L2_encoded=[]
L2_encoded.extend(L2_ypred)
L2_encoded.extend(L2_ytest_selected)
L2_encoded=list(set(L2_encoded))

confusion=metrics.confusion_matrix(L2_ytest_selected,L2_ypred)
plot_confusion_matrix(confusion,labels=L2_encoded,title='Layer2')




In [ ]:
L3_ytest_selected=L3_ytest.iloc[malicious_indices]

c1_model=joblib.load(os.path.join(saved_path,'class_1_CICI.pkl'))

indices1=np.where(L2_ypred==1)[0]

print('Reconnaissance train & test')

if indices1.any():
    c1_Xtest_selected=L2_Xtest.iloc[indices1]
    c1_ypred=c1_model.predict(c1_Xtest_selected)
    c1_ytest_selected=L3_ytest_selected.iloc[indices1]
    c1_result=test_result(c1_model,c1_ytest_selected,c1_ypred)
    model_eval.extend(c1_result)
else:
    model_eval.extend([0,0,0,0])

c2_model=joblib.load(os.path.join(saved_path,'class_2_CICI.pkl'))

indices2=np.where(L2_ypred==2)[0]

print('Access train & test ')

if indices2.any():
    c2_Xtest_selected=L2_Xtest.iloc[indices2]
    c2_ypred=c2_model.predict(c2_Xtest_selected)
    c2_ytest_selected=L3_ytest_selected.iloc[indices2]
    c2_result=test_result(c2_model,c2_ytest_selected,c2_ypred)
    
    model_eval.extend(c2_result)

else:
    model_eval.extend([0,0,0,0])

c3_model=joblib.load(os.path.join(saved_path,'class_3_CICI.pkl'))

indices3=np.where(L2_ypred==3)[0]
print('Dos train & test')

if indices3.any():
    c3_Xtest_selected=L2_Xtest.iloc[indices3]
    c3_ypred=c3_model.predict(c3_Xtest_selected)
    c3_ytest_selected=L3_ytest_selected.iloc[indices3]
    c3_result=test_result(c3_model,c3_ytest_selected,c3_ypred)

    
    model_eval.extend(c3_result)

else:
    model_eval.extend([0,0,0,0])

c4_model=joblib.load(os.path.join(saved_path,'class_4_CICI.pkl'))

indices4=np.where(L2_ypred==4)[0]
print('Malware train & test')
if indices4.any():
    c4_Xtest_selected=L2_Xtest.iloc[indices4]
    c4_ypred=c4_model.predict(c4_Xtest_selected)
    c4_ytest_selected=L3_ytest_selected.iloc[indices4]
    c4_result=test_result(c4_model,c4_ytest_selected,c4_ypred)
    
    model_eval.extend(c4_result)

else:
    model_eval.extend([0,0,0,0])






In [ ]:
final_y_pred=[]
for class_pred in [c1_ypred,c2_ypred,c3_ypred,c4_ypred]:
    final_y_pred.extend(class_pred)

final_y_test=[]
for class_test in [c1_ytest_selected,c2_ytest_selected,c3_ytest_selected,c4_ytest_selected]:
    final_y_test.extend(class_test)

final_result=test_result('Layer3',final_y_test,final_y_pred)
model_eval.extend(final_result)


In [ ]:
L3_encoded=[]
L3_encoded.extend(final_y_pred)
L3_encoded.extend(final_y_test)
L3_encoded=list(set(L3_encoded))

confusion=metrics.confusion_matrix(final_y_test,final_y_pred)
plot_confusion_matrix(confusion,labels=L3_encoded,title='Layer3')

In [ ]:
c1_encoded=[]
c1_encoded.extend(c1_ypred)
c1_encoded.extend(c1_ytest_selected)
c1_encoded=list(set(c1_encoded))

c2_encoded=[]
c2_encoded.extend(c2_ypred)
c2_encoded.extend(c2_ytest_selected)
c2_encoded=list(set(c2_encoded))

c3_encoded=[]
c3_encoded.extend(c3_ypred)
c3_encoded.extend(c3_ytest_selected)
c3_encoded=list(set(c3_encoded))

c4_encoded=[]
c4_encoded.extend(c4_ypred)
c4_encoded.extend(c4_ytest_selected)
c4_encoded=list(set(c4_encoded))

In [ ]:
# 개별 classifier에 대한 confusion matrix 
confusion=metrics.confusion_matrix(c1_ytest_selected,c1_ypred)
plot_confusion_matrix(confusion,labels=c1_encoded,title='Reconnaissance(5-way)')

confusion=metrics.confusion_matrix(c2_ytest_selected,c2_ypred)
plot_confusion_matrix(confusion,labels=c2_encoded,title='Access(9-way)')

confusion=metrics.confusion_matrix(c3_ytest_selected,c3_ypred)
plot_confusion_matrix(confusion,labels=c3_encoded,title='DoS(5-way)')

confusion=metrics.confusion_matrix(c4_ytest_selected,c4_ypred)
plot_confusion_matrix(confusion,labels=c4_encoded,title='Malware(3-way)')

In [ ]:
df.loc[cnt]=model_eval
df



In [ ]:
df.to_csv(os.path.join(outpath,'CICI_result.csv'))
